In [23]:
#Part II

#Data Preparation (From Part I)

import pandas as pd
import numpy as np

def parse_data(filename):
    A=[]
    text_file = open(filename, "r")
    lines = text_file.readlines()
    text_file.close()
    for i in range(0,len(lines)):
        id = lines[i].split(' ', 1)[0]
        string = lines[i].split(' ', 1)[1]
        rem_string = ''.join(e for e in string if e.isalnum()).lower()
        tupool = (id,rem_string)
        A.append(tupool)
    return A


In [24]:
#Shingle (From Part I)
import binascii

def shingle_document(string, k):
    word=string
    ShinglesHash = set()
    shingles = [word[i:i+k] for i in range(0, len(word)-k+1) if len(word[i])<k+1]
    for i in shingles:
        crc = binascii.crc32(str.encode(i)) & 0xffffffff
        ShinglesHash.add(crc)
    return ShinglesHash
    

In [25]:
#Jaccard (From Part I)
def jaccard(a, b):
    
    intersectionsize=len(a.intersection(b))
    unionsize=len(a.union(b))
    JS=intersectionsize/unionsize
    return JS
    

In [26]:

#Part Invert_shingles(From Part II)

def invert_shingles(shingled_documents):
    invert_item = []
    aIDs = []
    
    for (aID, shingles) in shingled_documents:
        aIDs.append(aID)
        for s in shingles:
            item = (s, aID)
            invert_item.append(item)
    results=sorted(invert_item),aIDs
    return results


In [27]:

#Make Random Hashes (From Part II)

import random

def make_random_hash_fn(p=2**33-355, m=4294967295):
    a = random.randint(1,p-1)
    b = random.randint(0, p-1)
    return lambda x: ((a * x + b) % p) % m


In [40]:

#Minhash Matrix (From Part II)


import numpy

def make_minhash_signature(shingled_data, num_hashes):
    make_hashes=make_random_hash_fn()
    inv_index, docids = invert_shingles(shingled_data)
    num_docs = len(docids)
  
  # initialize the signature matrix with infinity in every entry
    sigmatrix = np.full([num_hashes, num_docs], np.inf)
  
  # generate hash functions
    hash_funcs = make_hashes(num_hashes)
  
  # iterate over each non-zero entry of the characteristic matrix
    for row, docid in inv_index:
    # update signature matrix if needed 
        hashvals=[h(row) for h in hash_funcs]
        j=docids.index(docid)
    
        for i in range(num_hashes):
            if hashvals[i]<sigmatrix[i,j]:
                sigmatrix[i,j]=hashvals[i]
    # THIS IS WHAT YOU NEED TO IMPLEMENT
  
    return sigmatrix, docids




In [44]:
#Similarity_Minhash(From Part II)

def minhash_similarity(id1, id2, minhash_sigmat, docids):
    
    
    # get column of the similarity matrix for the two documents
    i=docids.index(id1)
    j=docids.index(id2)
    a=minhash_sigmat[:,i]
    b=minhash_sigmat[:,j]
            
            # calculate the fraction of rows where two columns match
    fraction = jaccard(a,b)
    
           
    # return this fraction as the minhash similarity estimate
    return fraction
  



In [ ]:
#Part III

#Choosing Number of Bands

import scipy.optimize as opt
import math

def _choose_nbands(t, n):
    def _error_fun(x):
        cur_t = (1/x[0])**(x[0]/n)
        return (t-cur_t)**2

    opt_res = opt.minimize(error_fun, x0=(10), method='Nelder-Mead')
    b = int(math.ceil(res['x'][0]))
    r = int(n / b)
    final_t = (1/b)**(1/r)
    return b, final_t


In [ ]:
# Hash a vector

def _make_vector_hash(num_hashes, m=4294967295):
    hash_fns = make_hashes(num_hashes)
    def _f(vec):
        acc = 0
        for i in range(len(vec)):
            h = hash_fns[i]
            acc += h(vec[i])
        return acc % m
    return _f

In [3]:
# LSH

from collections import defaultdict

def do_lsh(minhash_sigmatrix, numhashes, docids, threshold):
  # choose the number of bands, and rows per band to use in LSH
    b, _ = _choose_nbands(threshold, numhashes)
    r = int(numhashes / b)

    narticles = len(docids)
 
  # generate a random hash function that takes vectors of lenght r as input
    hash_func = _make_vector_hash(r)

  # setup the list of hashtables, will be populated with one hashtable per band
    buckets = []

  # fill hash tables for each band
    for band in range(b):
     # figure out which rows of minhash signature matrix to hash for this band
        start_index = int(band * r)
        end_index = min(start_index + r, numhashes)

    # initialize hashtable for this band
        cur_buckets = defaultdict(list)
    
        for j in range(narticles):
            # THIS IS WHAT YOU NEED TO IMPLEMENT
            hashe= minhash_sigmatrix[start_index:end_index, j] 
            hash_band_val = hash_func(hashe)
            dID = docids[j]
            cur_buckets[hash_band_val].append(dID)
      

    # add this hashtable to the list of hashtables
            buckets.append(cur_buckets)

    return buckets

In [ ]:
#Find candidate similar article pairs

import itertools

def pairs(buckets):
    results = []
    
    for i in buckets:
        for (k, SIDs) in i.items():
            sorIDs = sorted(SIDs)
            docs = list(itertools.combinations(sorIDs,2))
            for pair in doc_combinations:
                results.append(pair)
    
    return list(pairs)



In [4]:
# Main

import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plot

Location=r'C:\Users\yazdan\Desktop\MinHash-master\MinHash-master\data\articles_1000.train'

DATA = parse_data(Location)
toople=[]
results=[]
num_hashes=100
rmse=[]
threshold=0.8
k=10

for i in range(0,len(DATA)):
    a=shingle_document(DATA[i][1],10)
    aID=DATA[i][0]
    toople.append((aID,a))



sigmat,docids=make_minhash_signature(toople,num_hashes)
bucks=do_lsh(sigmat, numhashes, docids, threshold)
candidate=pair(bucks)



NameError: name 'parse_data' is not defined